In [2]:
import pandas as pd
import numpy as np
import os

import gymnasium # new version gymnasium
# from gym import Env -> old version gym
from gymnasium.spaces import Discrete, Dict, Sequence, Box, MultiBinary, MultiDiscrete

from enum import Enum

from stable_baselines3 import PPO

from stable_baselines3.common.env_checker import check_env 
# for chekcing if ur custom environment is compatible with stable baseline3 lib that 
# have the implementations for the model u are gonna use like ppo for example

# gonna remove it i guess
from stable_baselines3.common.callbacks import BaseCallback


# very important, should use another to get the best algo for our idea
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
df = pd.read_csv("C:/Users/AbdulRahman/Desktop/grad_folders/MAIN_GRAD/Travia/Implementation/AI/DataSet/lateset Dataset Main/pre_processed_df_with.csv")

In [5]:
df.head()

,latitude,longitude,stars,review_count,attributes_RestaurantsPriceRange2,attributes_GoodForKids,attributes_Ambience_touristy,attributes_Ambience_romantic,attributes_Ambience_intimate,attributes_Ambience_trendy,...,hours_Sunday,attributes_RestaurantsGoodForGroups,attributes_RestaurantsAttire,attributes_NoiseLevel,attributes_GoodForMeal_dessert,attributes_GoodForMeal_latenight,attributes_GoodForMeal_lunch,attributes_GoodForMeal_dinner,attributes_GoodForMeal_brunch,attributes_GoodForMeal_breakfast
0,39.955505,-75.155564,4.0,80,1,1,0,0,0,0,...,7:0-21:0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,39.953949,-75.143226,4.0,245,2,0,0,0,0,0,...,13:30-22:0,True,'casual',u'average',True,NaN,NaN,True,NaN,NaN
2,39.943223,-75.162568,4.5,205,1,1,0,0,0,0,...,NaN,False,u'casual',u'quiet',False,False,True,True,False,False
3,40.006707,-75.289671,3.5,109,1,1,0,0,0,0,...,11:0-0:0,True,u'casual',u'average',False,True,False,True,False,False
4,39.962582,-75.135657,3.5,65,2,1,0,0,0,0,...,11:0-20:0,NaN,NaN,NaN,False,False,False,False,False,False


In [8]:
not_pre_processed_df = pd.read_csv("C:/Users/AbdulRahman/Desktop/grad_folders/MAIN_GRAD/Travia/Implementation/AI/DataSet/Yelp Dataset/flattened_data.csv") 

C:\Users\AbdulRahman\AppData\Local\Temp\ipykernel_23152\712592108.py:1: DtypeWarning: Columns (9,18,39) have mixed types. Specify dtype option on import or set low_memory=False.
  not_pre_processed_df = pd.read_csv("C:/Users/AbdulRahman/Desktop/grad_folders/MAIN_GRAD/Travia/Implementation/AI/DataSet/Yelp Dataset/flattened_data.csv")


In [9]:
not_pre_processed_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,attributes_Ambience_touristy,attributes_Ambience_classy,attributes_Ambience_trendy,attributes_Ambience_casual,attributes_GoodForMeal_dessert,attributes_GoodForMeal_latenight,attributes_GoodForMeal_lunch,attributes_GoodForMeal_dinner,attributes_GoodForMeal_brunch,attributes_GoodForMeal_breakfast
0,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,...,False,False,False,False,False,False,False,False,False,False
1,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,False,False,False,False,False,False,False,False,False,False
2,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,...,False,False,False,False,False,False,False,False,False,False
3,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,"8522 Eager Road, Dierbergs Brentwood Point",Brentwood,MO,63144,38.627695,-90.340465,2.5,13,...,False,False,False,False,False,False,False,False,False,False
4,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,...,False,False,False,False,False,False,False,False,False,False


In [85]:
len(df.columns)

45

In [84]:
len(not_pre_processed_df.columns)

40

### price and stars range 

In [13]:
print(sorted(set(list(df['attributes_RestaurantsPriceRange2']))))

[1, 2, 3, 4]


In [14]:
print(sorted(set(list(df['stars']))))

[3.5, 4.0, 4.5, 5.0]


# Feature selection part

In [15]:
final_df = df[
    [
            "stars",
    
            "attributes_RestaurantsPriceRange2",
            
            # good for kids or romantic
            "attributes_GoodForKids", # option1
        
            "attributes_Ambience_romantic", # option2
            "attributes_Ambience_intimate", # option2
        
            "attributes_Ambience_touristy",
        
            # trendy or classy or casual 
            "attributes_Ambience_trendy",
            "attributes_Ambience_classy",
            "attributes_Ambience_casual",
        
            # choose one or many from those 
            "Bars_Night",
            "Beauty_Health_Care",
            "Cafes",
            "GYM",
            "Restaurants_Cuisines",
            "Shops"
    ]
]

In [16]:
final_df.head()

,stars,attributes_RestaurantsPriceRange2,attributes_GoodForKids,attributes_Ambience_touristy,attributes_Ambience_romantic,attributes_Ambience_intimate,attributes_Ambience_trendy,attributes_Ambience_classy,attributes_Ambience_casual,Bars_Night,Beauty_Health_Care,Cafes,GYM,Restaurants_Cuisines,Shops
0,4.0,1,1,0,0,0,0,0,1,0,0,1,0,1,0
1,4.0,2,0,0,0,0,0,1,0,0,0,0,0,1,0
2,4.5,1,1,0,0,0,0,0,1,0,0,0,0,1,0
3,3.5,1,1,0,0,0,0,0,1,0,0,0,0,1,0
4,3.5,2,1,0,0,0,0,1,0,0,0,0,0,1,0


In [ ]:
final["attributes_RestaurantsPriceRange2"] -= 1

# Building the enviroment 

In [ ]:
class Travia(gymnasium.Env):
    # parameters for just testing
    # use_static_prefs is for teseting
    # original df is for testing in render too 
    # user pref
    def __init__(self,final_dataset,original_df,user_pref):
        super().__init__()
        # if we did not use super.init() and u forgot to write some properties (action_space, observation_space) 
        # that env can not work and will generate an error
        
        self.user_pref = user_pref
        self.original_df = original_df
        self.businesses = np.array(data, dtype=np.float32)

        self.num_features = 15
        self.category_cols = [
            "Bars_Night", "Beauty_Health_Care", "Cafes", "GYM",
            "Restaurants_Cuisines", "Shops"
        ]



        # if we used this option1: 
        #     self.observation_space = Dict({
        #     "user_pref": Dict({
        #         "category": Discrete(6),
        #         "min_stars": Box(low=0.0, high=5.0, shape=(1,), dtype=np.float32),
        #         "price_range": Discrete(4),
        #         "attributes_Ambience_touristy": Discrete(2),
        #         "attributes_Ambience_trendy": Discrete(2),
        #         "attributes_Ambience_classy": Discrete(2),
        #         "attributes_Ambience_casual": Discrete(2),
        #         "intimate_or_family": Discrete(2)
        #     }),
        #     "business_space": Dict({
        #         "stars": Box(low=0, high=5, shape=(1,), dtype=np.float32),
        #         "attributes_RestaurantsPriceRange2": Discrete(5),
        #         "attributes_GoodForKids": Discrete(2),
        #         "attributes_Ambience_touristy": Discrete(2),
        #         "attributes_Ambience_romantic": Discrete(2),
        #         "attributes_Ambience_intimate": Discrete(2),
        #         "attributes_Ambience_trendy": Discrete(2),
        #         "attributes_Ambience_classy": Discrete(2),
        #         "attributes_Ambience_casual": Discrete(2),
        #         "Bars_Night": Discrete(2),
        #         "Beauty_Health_Care": Discrete(2),
        #         "Cafes": Discrete(2),
        #         "GYM": Discrete(2),
        #         "Restaurants_Cuisines": Discrete(2),
        #         "Shops": Discrete(2),
        #     })
        # })
        # there will be a problem in policy network’s input processing
        
        # option 2
        n_businesses = len(data)
        n_columns = 15
        self.observation_space = Dict({
            "category": Discrete(6),
            "min_stars": Box(low=0.0, high=5.0, shape=(1,), dtype=np.float32),
            "price_range": Discrete(4),  # 0–3 (mapped from 1–4)
            "attributes_Ambience_touristy": Discrete(2),
            "attributes_Ambience_trendy": Discrete(2),
            "attributes_Ambience_classy": Discrete(2),
            "attributes_Ambience_casual": Discrete(2),
            "intimate_or_family": Discrete(2),
            "business_data": Box(
                # "stars", "attributes_RestaurantsPriceRange2", "attributes_GoodForKids", "attributes_Ambience_touristy", "attributes_Ambience_romantic", "attributes_Ambience_intimate", "attributes_Ambience_trendy", "attributes_Ambience_classy", "attributes_Ambience_casual", "Bars_Night", "Beauty_Health_Care", "Cafes", "GYM", "Restaurants_Cuisines", "Shops"
                low=np.tile(np.array([0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=np.float32), (n_businesses, 1)),
                high=np.tile(np.array([5.0, 3.0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=np.float32), (n_businesses, 1)),
                shape=(n_businesses, n_columns),
                dtype=np.float32
            )
        })
        self.action_space = Box(
            low=-1.0,
            high=1.0,
            shape=(self.num_features,),
            dtype=np.float32
        )
        self.selected_indices = []
    
    
    def get_obs(self):
        return {
            "category": self.user_pref["category"],
            "min_stars": np.array([self.user_pref["min_stars"]], dtype=np.float32),
            "price_range": self.user_pref["price_range"],
            "attributes_Ambience_touristy": self.user_pref["attributes_Ambience_touristy"],
            "attributes_Ambience_trendy": self.user_pref["attributes_Ambience_trendy"],
            "attributes_Ambience_classy": self.user_pref["attributes_Ambience_classy"],
            "attributes_Ambience_casual": self.user_pref["attributes_Ambience_casual"],
            "intimate_or_family": self.user_pref["intimate_or_family"],
            "business_data": self.businesses
        }
    def convert_user_pref_to_vector(self, user_pref):
            # i expect the uesr_pref to be a map
            user_pref_vector = np.zeros(self.num_features, dtype=np.float32)

            # map user_pref to the corresponding indices in feature_cols
            user_pref_vector[0] = user_pref["min_stars"]
            user_pref_vector[1] = user_pref["price_range"]
            
            if user_pref["intimate_or_family"] == 0:  # intimate/romantic
                user_pref_vector[4] = 1  # attributes_Ambience_romantic
                user_pref_vector[5] = 1  # attributes_Ambience_intimate
            elif user_pref["intimate_or_family"] == 1:  # family-friendly
                user_pref_vector[2] = 1  # attributes_GoodForKids

            user_pref_vector[3] = user_pref["attributes_Ambience_touristy"]  # touristy
            user_pref_vector[6] = user_pref["attributes_Ambience_trendy"]    # trendy
            user_pref_vector[7] = user_pref["attributes_Ambience_classy"]    # classy
            user_pref_vector[8] = user_pref["attributes_Ambience_casual"]    # casual

            category_idx = user_pref["category"]  # 0 to 5
            user_pref_vector[9 + category_idx] = 1

            return np.array(user_pref_vector)

    def sample_user_pref(self, seed=None):
        if seed is not None:
            np.random.seed(seed)
        return {
            "min_stars": np.random.uniform(3.5, 5.0), # 3-5
            "price_range": np.random.randint(0, 4),  # 0–3
            "intimate_or_family": np.random.randint(0, 2),
            "attributes_Ambience_touristy": np.random.randint(0, 2),
            "attributes_Ambience_trendy": np.random.randint(0, 2),
            "attributes_Ambience_classy": np.random.randint(0, 2),
            "attributes_Ambience_casual": np.random.randint(0, 2),
            "category": np.random.randint(0, 6)
        }

#     def calc_reward(self,selected_businesses):
#         target_vector =  self.convert_user_pref_to_vector(self.user_pref).reshape(1, -1)
#         similarities = cosine_similarity(np.array(selected_businesses), target_vector).flatten()
#         reward = np.mean(similarities)
#         stars = selected_businesses[:, 0]
#         if np.any(stars < self.user_pref["min_stars"]):
#             reward -= 0.1 * np.sum(stars < self.user_pref["min_stars"])
#         return float(np.clip(reward, 0, 1))
    def calc_reward(self, selected_businesses):
        target_vector = self.convert_user_pref_to_vector(self.user_pref).reshape(1, -1)
        similarities = cosine_similarity(selected_businesses, target_vector).flatten()
        reward = np.mean(similarities)
        stars = selected_businesses[:, 0]
        if np.any(stars < self.user_pref["min_stars"]):
            reward -= 0.1 * np.sum(stars < self.user_pref["min_stars"])
        clipped_reward = float(np.clip(reward, 0, 1))
        # print(f"Raw Reward: {reward}, Clipped: {clipped_reward}")  # debug
        return clipped_reward


    def reset(self, seed=None):
        if seed is not None:
            np.random.seed(seed)
        self.selected_indices = []
        if not self.use_static_prefs or self.user_pref is None:
            self.user_pref = self.sample_user_pref(seed)
        return self.get_obs(), {}
    
    def step(self, action, k=3):
        business_vectors = np.array(self.businesses)
        similarities = cosine_similarity(business_vectors, action.reshape(1, -1)).flatten()
        top_k_indices = np.argpartition(similarities, -k)[-k:]
        self.selected_indices = top_k_indices.tolist()
        selected_businesses = business_vectors[top_k_indices]
        
        reward = self.calc_reward(selected_businesses)
        obs = self.get_obs()

        terminated = bool(reward > 0.9)  
        truncated = terminated  # No time limit
#         print(f"Debug: reward={reward}, terminated={terminated}, type={type(terminated)}") 
        return obs, reward, terminated, truncated, {}
    
    def render(self, mode="human"):
        if mode == "human" and self.selected_indices:
            print(f"User Preferences: {self.user_pref}")
            print(f"Selected Businesses (Indices: {self.selected_indices}):")
            if self.original_df is not None and not self.original_df.empty:
                selected_rows = self.original_df.iloc[self.selected_indices]
                for idx, row in zip(self.selected_indices, selected_rows.itertuples()):
                    print(f"  Index {idx}:")
                    print(f"    Name: {row.name}")
                    print(f"    Stars: {row.stars}")
                    print(f"    Price Range: {row.attributes_RestaurantsPriceRange2+1}")
                    print(f"    Address: {row.address}, {row.city}, {row.state} {row.postal_code}")
                    print(f"    Category: {self.category_cols[np.argmax(row[-6:])]}")
                    print(f"    Good for Kids: {row.attributes_GoodForKids}")
                    print(f"    Ambiance: Trendy={row.attributes_Ambience_trendy}, Classy={row.attributes_Ambience_classy}, Casual={row.attributes_Ambience_casual}")
            else:
                print("  (No original DataFrame provided)")
            reward = self.calc_reward(np.array(self.businesses)[self.selected_indices])
            print(f"Reward: {reward:.4f}")
#     def render(self, mode="human"):
#         if mode == "human" and self.selected_indices:
#             print(f"User Preferences: {self.user_pref}")
#             print(f"Selected Businesses (Indices: {self.selected_indices}):")
#             for idx in self.selected_indices:
#                 business = self.businesses[idx]
#                 print(f"  Index {idx}: Stars={business[0]:.1f}, Price={business[1]:.1f}, Features={business[2:]}")
#             print(f"Reward: {self.calc_reward(np.array(self.businesses)[self.selected_indices]):.4f}")